In [1]:
from binance.client import Client

# support python <=3.9

ImportError: cannot import name 'COMMON_SAFE_ASCII_CHARACTERS' from 'charset_normalizer.constant' (D:\ProgramData\anaconda3\envs\google_ai\lib\site-packages\charset_normalizer\constant.py)

In [2]:
from binance.client import Client
from dotenv import dotenv_values
from datetime import datetime
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import sqlite3
import requests

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [11]:
is_py=False
is_new_entry=False
x_cryptocompare='y'

path_config=r'D:\AB_DB\Script_ImportData'
path_data=path_config+"\\Crypto_File"
db_path=path_config+"\\Database\\crypto_db.db"

today_now=datetime.now()
timeframe="4h" 

buildtime = datetime.now().strftime('%d%m%y_%H%M')

print(path_config)
print(path_data)
print(db_path)


D:\AB_DB\Script_ImportData
D:\AB_DB\Script_ImportData\Crypto_File
D:\AB_DB\Script_ImportData\Database\crypto_db.db


In [12]:
cols=["datetime","open","high","low","close","volume"]
cols_number=["open","high","low","close","volume"]
cols_last=["datetime","open","high","low","close","volume"]

cols_crypto_ohlc=["datetime","open","high","low","close",]
cols_crypto_vol=["datetime","volume"]

In [13]:
config_values= dotenv_values(dotenv_path=f'{path_config}\\.env')
api_key = config_values['bn_key']
api_secret =config_values['bn_secrete']
client=Client(api_key,api_secret)



In [14]:
cryptocompare_key=config_values['cryptocompare_key']
headers = {'Authorization': 'Bearer ' + cryptocompare_key}

In [15]:

ok=False

if is_py:
    print("Enter start date as yyyy-mm-dd(2019-01-01)")
    start_param = input("Enter start : ")

    print("Enter end date as yyyy-mm-dd(2022-01-01) if today ,press enter")
    end_param = input("Enter end : ") or ''
    
    print("Load Cryptocompare y|n")
    x_cryptocompare = input("Enter: ") or 'y'

    start=start_param
    if end_param == '':
     end=today_now.strftime("%Y-%m-%d")
    else:
     end=end_param  


    try:
        start_dt=datetime.strptime(start, "%Y-%m-%d")
        end_dt=datetime.strptime(end,"%Y-%m-%d")
        if start_dt>=end_dt:
           raise Exception(f"Error : {start_dt} can't be greater or equal to {end_dt}")
    except Exception as ex:
        raise ex



    print(f"Do you want to retrive data from {start_dt} to {end_dt} ?")
    press_y=input(f"Press y=True and n=False : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()
else: 
    start='2022-08-10'
    end= today_now.strftime("%Y-%m-%d")
    
limit_days=(( datetime.strptime(end,"%Y-%m-%d") )- ( datetime.strptime(start, "%Y-%m-%d") )).days 
limit_days=limit_days+1
    
print(f"{start} - {end}={limit_days}")

2022-08-10 - 2022-08-16=7


In [16]:
print("Connect SQLite")
conn = sqlite3.connect(os.path.abspath(db_path))


Connect SQLite


In [18]:

if is_new_entry:

    sql="select * from crypto_meta_info where is_new = 1 and api_source='binance'"
    sql_cpx="select * from crypto_meta_info where  is_new=1  and api_source='cryptocompare'"
else:

    sql="select * from crypto_meta_info where  is_new=0 and is_active=1 and api_source='binance'"
    sql_cpx="select * from crypto_meta_info where  is_new=0 and is_active=1 and api_source='cryptocompare'"

sql_valid=False
try:
 print("Load Binance")
 
 print(sql)
 print(sql_cpx)
 
 dfCryptos_MetaData = pd.read_sql_query(sql, conn)
 dfCryptos_MetaData=dfCryptos_MetaData[['name','symbol','category','pair','api_source']]   
 print(dfCryptos_MetaData.info())
 print(dfCryptos_MetaData.head())   
    
    
 print("Load Cryptocompare")
 dfCompare_MetaData = pd.read_sql_query(sql_cpx, conn)  
 dfCompare_MetaData=dfCompare_MetaData[['name','symbol','category','pair','all_volume','api_source']]
 print(dfCompare_MetaData.info())
 print(dfCompare_MetaData.head())   


 sql_valid=True 

except Exception as ex:
 raise  ex




Load Binance
select * from crypto_meta_info where  is_new=0 and is_active=1 and api_source='binance'
select * from crypto_meta_info where  is_new=0 and is_active=1 and api_source='cryptocompare'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        39 non-null     object
 1   symbol      39 non-null     object
 2   category    39 non-null     object
 3   pair        39 non-null     object
 4   api_source  39 non-null     object
dtypes: object(5)
memory usage: 1.6+ KB
None
       name symbol    category  pair api_source
0   Bitcoin    BTC  Blockchain  BUSD    binance
1  Ethereum    ETH  Blockchain  BUSD    binance
2   Cardano    ADA  Blockchain  BUSD    binance
3  Polkadot    DOT  Blockchain  BUSD    binance
4    Solana    SOL  Blockchain  BUSD    binance
Load Cryptocompare
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
D

# Binance API

In [19]:
list_not_found=[]
# get data from Binance API
for index,item in  dfCryptos_MetaData.iterrows():
  try:  
    symbol=item['symbol']
    stablecoin=item['pair']
    name=item['name']
    product=f"{symbol}{stablecoin}"
    # check bn_symbols.txt
    print(f"{symbol} : {start} and {end}")
    df= pd.DataFrame(client.get_historical_klines(product,timeframe,start,end))
    
    df_ohlcv=df.iloc[:,:6]
    df_ohlcv.columns=cols

    df_ohlcv["datetime"]=pd.to_datetime(df_ohlcv["datetime"],unit="ms")
    df_ohlcv[cols_number]=df_ohlcv[cols_number].astype("float")
    df_ohlcv=df_ohlcv[cols_last]
    
    print(df_ohlcv.head())
    print(df_ohlcv.tail())
    
    file_data=f'{symbol}.csv'
    path_file=os.path.join(path_data,file_data)
    df_ohlcv.to_csv(path_file, header=True, index=False)
    #file_data=f'{symbol}.txt'
    #df_ohlcv.to_csv(path_file, header=True, index=False, sep=',', mode='a')
   

    
  except Exception as ex:
    print(f'{symbol}: {ex}')
    list_not_found.append(f'{symbol} - {name}')
    
print('list not found symbol')
for item in list_not_found:
 print(item)

BTC : 2022-08-10 and 2022-08-16
             datetime      open      high       low     close       volume
0 2022-08-10 00:00:00  23157.17  23157.74  22666.66  22869.22  10141.48381
1 2022-08-10 04:00:00  22869.22  23019.60  22835.38  22971.60   5683.13025
2 2022-08-10 08:00:00  22971.72  23210.15  22971.56  23129.78   7562.85061
3 2022-08-10 12:00:00  23129.78  24196.97  23015.17  24004.84  28172.85082
4 2022-08-10 16:00:00  24001.51  24098.44  23483.69  23643.41  11087.41576
              datetime      open      high       low     close       volume
32 2022-08-15 08:00:00  24032.71  24320.48  23880.00  24058.25  13121.34192
33 2022-08-15 12:00:00  24055.83  24270.22  23906.17  24233.28  15210.66473
34 2022-08-15 16:00:00  24234.38  24287.07  23970.43  23984.79  10641.16017
35 2022-08-15 20:00:00  23984.18  24197.63  23776.50  24096.85  10514.92453
36 2022-08-16 00:00:00  24096.89  24251.13  23860.85  24088.00  12631.65167
ETH : 2022-08-10 and 2022-08-16
             datetime     open

# Cryptocompare API

In [56]:
if x_cryptocompare=='y':

    x_not_found=[]
    for index,item in  dfCompare_MetaData.iterrows():

     try:  
        name=item['name']
        # name='BTC'

        symbol=item['symbol']

        pair=item['pair']
        required_all_vols=item['all_volume']

        url = f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={name}&tsym={pair}&limit={limit_days}"
        print(url)
        json_ohlc=requests.get(url, headers=headers).json()

        df_ohlc=pd.DataFrame(json_ohlc['Data']['Data'])
        df_ohlc=df_ohlc.query('close>0')
        df_ohlc['datetime'] = [datetime.fromtimestamp(d) for d in df_ohlc.time]
        if required_all_vols==1:

            df_ohlc=df_ohlc[cols_crypto_ohlc]
            print("Require all volume")
            url_vol=f"https://min-api.cryptocompare.com/data/symbol/histoday?fsym={name}&tsym={pair}&limit={limit_days}"
            print(url_vol)
            json_vol=requests.get(url_vol, headers=headers).json()   

            df_vol=pd.DataFrame(json_vol['Data'])
            df_vol['datetime'] = [datetime.fromtimestamp(d) for d in df_vol.time]
            df_vol=df_vol.drop(columns=['time'])
            df_vol=df_vol[['datetime','total_volume_total']]
            df_vol=df_vol.rename(columns={'total_volume_total':'volume'})

            print(df_vol.info())
            print(df_vol.head())

            df_cypto_compare=pd.concat([df_ohlc,df_vol['volume']],axis=1)

        elif required_all_vols==0:

          df_ohlc=df_ohlc.rename(columns={'volumeto':'volume'})
          df_cypto_compare=df_ohlc[cols_crypto_ohlc+['volume']]

        print(df_cypto_compare.info())
        print(df_cypto_compare.head())


        xfile_data=f'{symbol}.csv'
        xpath_file=os.path.join(path_data,xfile_data)
        df_cypto_compare.to_csv(xpath_file, header=True, index=False)

     except Exception as ex:
          print(f'{symbol}: {ex}')
          x_not_found.append(f'{symbol} - {name}')



https://min-api.cryptocompare.com/data/v2/histoday?fsym=APE&tsym=USD&limit=856
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 807 to 856
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  50 non-null     datetime64[ns]
 1   open      50 non-null     float64       
 2   high      50 non-null     float64       
 3   low       50 non-null     float64       
 4   close     50 non-null     float64       
 5   volume    50 non-null     float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 2.7 KB
None
               datetime   open   high     low  close       volume
807 2022-03-17 07:00:00  10.60  10.60  10.600  10.60   3245178.31
808 2022-03-18 07:00:00  10.60  10.60  10.080  10.08  38533693.34
809 2022-03-19 07:00:00  10.08  10.08  10.080  10.08         0.00
810 2022-03-20 07:00:00  10.08  10.08  10.080  10.08         0.00
811 2022-03-21 07:00:00  10.08  10.82   9.749  10.53  